A simple programme that apply momentum investing strategy,
Free sandbox data is used and is supplyed by IEX cloud API .

The programme screen through stocks in S&P500 and filter out top 20 stocks with greatest 1 year price return. Then number of stocks to buy is calculated for that list after inputing portfolio size

In [3]:
import numpy as np
import pandas as pd
import requests 
import math
from scipy import stats

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [5]:
#checking the api call
symbol = "AAPL"
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#data

In [6]:
# Define a function for making sublists
def chunks(lst,stepsize):
    for i in range(0, len(lst), stepsize):
        yield lst[i:i+stepsize]
        

In [21]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range (0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
column_names = ['Ticker','Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [22]:
# Batch api requests
final_df = pd.DataFrame(columns = column_names)

for symbol_string in symbol_strings :
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append( pd.Series([symbol,
                                               data[symbol]['quote']['latestPrice'],
                                               data[symbol]['stats']['year1ChangePercent'],
                                               'N/A'],
                                               index = column_names),
                                               ignore_index = True) 

        
        
#final_df

In [41]:
# Remove low-momentum stocks

final_df.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_df = final_df[:20]
final_df.reset_index(drop = True, inplace = True)
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,39.91,3.95677,99
1,MRO,16.50,3.25022,240
2,FANG,108.74,3.19378,36
3,OXY,34.50,2.78544,114
4,LB,80.87,2.30906,48
5,EOG,94.75,1.8311,41
6,MCHP,74.30,1.73143,53
7,COTY,8.35,1.72483,474
8,COP,76.27,1.6837,51
9,IT,322.41,1.68239,12


In [42]:
# A basic method to calculate no. of shares to buy

def portfolio():
    global portfolio_size
    portfolio_size = input('Size of the portfolio = ')
portfolio()    

Size of the portfolio = 1000000


In [43]:
size = float(portfolio_size)/len(final_df.index)
for i in range (0,len(final_df['Ticker'])):
    final_df.loc[i,'Number of Shares to Buy'] = math.floor(size/final_df['Price'][i])
    
final_df

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,39.91,3.95677,1252
1,MRO,16.50,3.25022,3030
2,FANG,108.74,3.19378,459
3,OXY,34.50,2.78544,1449
4,LB,80.87,2.30906,618
5,EOG,94.75,1.8311,527
6,MCHP,74.30,1.73143,672
7,COTY,8.35,1.72483,5988
8,COP,76.27,1.6837,655
9,IT,322.41,1.68239,155
